Installs **ONNX** to switch from a TF model to an ONNX model.
The ONNX model is designed for sharing and to be more global.

In [ ]:
!pip install onnxmltools onnx onnxruntime
!pip install -U tf2onnx

Import required packages to train model.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pathlib

from tensorflow import keras
from tensorflow.keras import layers

import onnxmltools
import tf2onnx

In [ ]:
print("Tensorflow version:", tf.version.VERSION)
print("Keras version:", keras.__version__)

Set the path of the dataset.

In [ ]:
data_dir = pathlib.Path(r"/content/drive/MyDrive/datasets/nude_or_not")

Define variables for train the model.

In [ ]:
IMG_SIZE = (224, 224) # MobileNetV3 standard image size.
NUM_CLASSES = 1
LEARNING_RATE = 1e-4
DROPOUT_RATE = 0.5 # avoid overfitting for little dataset.
BATCH_SIZE = 64
SEED = 64

Split dataset for train & validation.

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
  data_dir,
  label_mode="binary",
  validation_split=0.2,
  subset="training",
  seed=SEED,
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE)

val_ds = keras.utils.image_dataset_from_directory(
  data_dir,
  label_mode="binary",
  validation_split=0.2,
  subset="validation",
  seed=SEED,
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE)

Define training and validating variables.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Feature extraction.

In [ ]:
from tensorflow.keras.applications import MobileNetV3Small

IMG_SHAPE = IMG_SIZE + (3,)
base_model = MobileNetV3Small(
  weights="imagenet",
  include_top=False,
  input_shape=IMG_SHAPE,
)

# Freeze weights for feature extractor.
base_model.trainable = False

Data augmentation.

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=IMG_SHAPE),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

Custom Keras model.

In [ ]:
class NudeDetectorModel(keras.Model):
  def __init__(self, batch_size=None, trainable=False):
    """Create custom NudeDetector model using Keras for feature extraction."""
    batch_shape = (batch_size,) + IMG_SHAPE

    inputs = keras.Input(batch_shape=batch_shape, name="input")

    x = data_augmentation(inputs, training=trainable)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(DROPOUT_RATE)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(DROPOUT_RATE)(x)
    outputs = layers.Dense(NUM_CLASSES, activation="sigmoid", name="output")(x)

    super().__init__(inputs=inputs, outputs=outputs, name="nude_detector")

In [ ]:
model = NudeDetectorModel(trainable=True)

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

model.summary()

In [ ]:
EPOCHS = 6
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=EPOCHS,
  batch_size=BATCH_SIZE,
)

Fine tuning.

In [ ]:
base_model.trainable = True
base_model_layers = len(base_model.layers)
base_model_layers

We only train last 50 layers to adjust specific traits.

---



In [ ]:
FINE_TUNE_LAYER = base_model_layers - 30
for layer in base_model.layers[:FINE_TUNE_LAYER]:
  layer.trainable = False
FINE_TUNE_LAYER

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE/10),
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

model.summary()

In [ ]:
len(model.trainable_variables)

In [ ]:
FINE_TUNE_EPOCHS = 6
TOTAL_EPOCHS =  EPOCHS + FINE_TUNE_EPOCHS

history_fine = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=TOTAL_EPOCHS,
  initial_epoch=history.epoch[-1],
  batch_size=BATCH_SIZE,
)

Show graph of training

In [ ]:
acc = history.history["accuracy"] + history_fine.history["accuracy"]
val_acc = history.history["val_accuracy"] + history_fine.history["val_accuracy"]
auc = history.history["auc"] + history_fine.history["auc"]

loss = history.history["loss"] + history_fine.history["loss"]
val_loss = history.history["val_loss"] + history_fine.history["val_loss"]
TOTAL_EPOCHS = len(acc)
epochs_range = range(TOTAL_EPOCHS)

EPOCHS_FEATURE_EXTRACTOR = len(history.history["accuracy"])

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_acc, label="Validation Accuracy")

plt.plot(epochs_range, auc, label="Area Under the Curve")

plt.ylim([0, 1])

plt.plot([EPOCHS_FEATURE_EXTRACTOR - 1, EPOCHS_FEATURE_EXTRACTOR - 1],
          plt.ylim(), label="Starting Fine Tuning", linestyle='--', color='r')
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")

plt.plot([EPOCHS_FEATURE_EXTRACTOR - 1, EPOCHS_FEATURE_EXTRACTOR - 1],
          plt.ylim(), label="Starting Fine Tuning", linestyle='--', color='r')
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")

plt.show()

Convert TF model to ONNX model and save it.

In [ ]:
weights = model.get_weights()
model_fixed = NudeDetectorModel(batch_size=1, trainable=True)
model_fixed.set_weights(weights)

In [ ]:
MODEL_NAME = "corpus-{}.onnx"

In [ ]:
model_fixed.output_names=["output"]

In [ ]:
input_signature = [tf.TensorSpec(
  model_fixed.inputs[0].shape,
  model_fixed.inputs[0].dtype,
  name="input")]

In [ ]:
onnx_model, _ = tf2onnx.convert.from_keras(model_fixed, input_signature, opset=18)
onnxmltools.utils.save_model(onnx_model, MODEL_NAME.format("large"))

In [ ]:
from onnxruntime.quantization import shape_inference

shape_inference.quant_pre_process(
  MODEL_NAME.format("large"), MODEL_NAME.format("infer")
)

In [ ]:
from onnxruntime.quantization import *

quantize_dynamic(
  MODEL_NAME.format("infer"),
  MODEL_NAME.format("small-dynamic"),
  weight_type=QuantType.QUInt8,
)

You finished the training. See generated models.
* `large`: not quantified (FLOAT32);
* `small-dynamic`: quantified model (UNIT8).